# 4. 評価メトリクスとコールバック


<style>
pre {
    border: 1px solid #333;
    padding: 20px;
    margin: 20px 0;
    background-color: #000000;
    color: #d4d4d4;
    border-radius: 8px;
}
pre code {
    color: #d4d4d4;
    display: block;
    padding-bottom: 8px;
    background-color: #000000; 
}

.hljs, .language-python {
    background-color: #000000 !important;
}
</style>

<div style="background-color: #F9F4F0; padding: 10px; border-left: 5px solid #4CAF50; margin: 10px; width: 95%;">
    <details>
        <summary style="color: #8A6F5C; font-size: 1.17em; font-weight: bold;">claude解説</summary>
        <div style="color: #8A6F5C;">

この部分のコードについて、わかりやすく説明させていただきます。

# 評価メトリクスとコールバックの主な役割

このセクションは、AIモデルの学習過程を監視し、モデルがどれだけ「ソクラテス式の対話」をうまく再現できているかを評価する部分です。大きく分けて3つの主要な機能があります：

## 1. 文章スタイルの評価 (compute_metrics関数)

この関数は、モデルが生成した文章が「ソクラテス式の対話」らしさを持っているかを評価します。

例えば：
```python
sentence_end_patterns = {
    'question_patterns': [
        'かね', 'だろうか', 'ではないか',
        'のか', 'と思わないか', '考えてみよう',
    ]
}
```

これは、以下のような文末表現を探します：
- 「それについてどう考えるかね？」
- 「その考えは正しいのだろうか？」
- 「別の視点から見てみてはどうだろうか？」

このような表現がどれだけ適切に使われているかをスコア化します。

## 2. 対話の流れの評価 (calculate_dialogue_flow関数)

対話の自然な流れを評価します。具体的には：

1. 質問と説明のバランス（理想は30%が質問文）
2. 文の長さの変化（一定の長さに偏りすぎていないか）
3. 接続詞の使用（「しかし」「だから」などで文章をつなげているか）

例えば：
```text
ユーザー: 幸せとは何だと思いますか？
AI: なるほど、幸せの定義について考えてみましょう。
まず、あなたにとって幸せとは何でしょうか？
そして、その幸せは誰もが共有できるものだと思いますか？
```

このような対話の流れが自然かどうかを評価します。

## 3. システムリソースの監視 (TrainingMonitorCallback)

学習中のコンピュータの状態を監視します：
- メモリ使用量
- GPU使用率
- 学習の進行状況
- エラーの検出

これにより、学習が正常に進んでいるか、問題が発生していないかを確認できます。

## まとめ

このコードは、単にAIモデルを学習させるだけでなく、「ソクラテス式の対話」という特殊な対話スタイルをどれだけ正確に再現できているかを細かくチェックし、その品質を保証する役割を果たしています。また、学習中のシステムの健全性も同時に監視することで、安定した学習プロセスを実現しています。

        
</div>
    </details>
</div>


In [ ]:


# Update evaluation metrics
def compute_metrics(eval_preds):
    logits, labels = eval_preds  # Get logits and labels from eval_preds
    
    # Relax size limit for evaluation dataset
    max_samples = 100
    
    # Improve decoding process
    with torch.no_grad():
        logits = torch.tensor(logits).cpu()
        predictions = torch.argmax(logits, dim=-1)
        
        # Decode batch
        decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
        
        # Add more detailed logging
        logging.info(f"Sample prediction: {decoded_preds[0][:100]}...")
        
        del logits, predictions  # Memory release
        torch.cuda.empty_cache()
        
        # Define sentence ending patterns more flexibly
        sentence_end_patterns = {
            'question_patterns': [
                'かね', 'だろうか', 'ではないか',
                'のか', 'と思わないか', '考えてみよう',
            ],
            'statement_patterns': [
                'だね', 'なるほど', '興味深い',
                'といえよう', 'というべきだ'
            ],
            'reflection_patterns': [
                'かもしれない', 'のではないか',
                'と考えられる', 'といえそうだ'
            ]
        }
        
        # Auxiliary verb patterns
        auxiliary_patterns = [
            'である', 'だ', 'です', 'ます',
            'のだ', 'のです', 'のである'
        ]
        
        def calculate_style_consistency(text):
            sentences = text.split('。')
            if not sentences:
                return 0.0
                
            # Evaluate sentence ending style consistency
            end_style_scores = []
            for sent in sentences:
                if not sent.strip():
                    continue
                    
                # Evaluate sentence ending patterns (partial match allowed)
                pattern_found = False
                for pattern_type, patterns in sentence_end_patterns.items():
                    if any(p in sent[-10:] for p in patterns):  # Search within 10 characters at the end
                        pattern_found = True
                        break
                end_style_scores.append(1.0 if pattern_found else 0.0)
            
            # Evaluate auxiliary verb consistency
            aux_style_scores = []
            for sent in sentences:
                if not sent.strip():
                    continue
                    
                # Evaluate auxiliary verb usage in the sentence
                aux_found = any(p in sent for p in auxiliary_patterns)
                aux_style_scores.append(1.0 if aux_found else 0.0)
            
            # Evaluate sentence length consistency
            lengths = [len(s.strip()) for s in sentences if s.strip()]
            length_variance = np.var(lengths) if lengths else 0
            length_score = 1.0 / (1.0 + length_variance/100)  # Higher score if variance is small
            
            # Overall evaluation
            end_style_avg = np.mean(end_style_scores) if end_style_scores else 0
            aux_style_avg = np.mean(aux_style_scores) if aux_style_scores else 0
            
            # Weighting
            weights = {
                'end_style': 0.5,
                'aux_style': 0.3,
                'length_consistency': 0.2
            }
            
            return (
                weights['end_style'] * end_style_avg +
                weights['aux_style'] * aux_style_avg +
                weights['length_consistency'] * length_score
            )
        
        # Evaluate style consistency for each prediction
        style_scores = [calculate_style_consistency(pred) for pred in decoded_preds]
        
        # Evaluate dialogue flow
        def calculate_dialogue_flow(text):
            sentences = text.split('。')
            if not sentences:
                return 0.0
            
            # 質問文判定の改善
            question_markers = {
                'explicit': ['？', '?'],  # 明示的な質問符号
                'patterns': [
                    'かね', 'だろうか', 'ではないか', 'のか', 
                    'と思わないか', '考えてみよう',
                    'どう', 'いかが', 'なぜ', 'どのように',
                    '問', '教えて', '聞かせて'
                ]
            }
            
            def is_question(sentence):
                # 明示的な質問符号のチェック
                if any(marker in sentence for marker in question_markers['explicit']):
                    return True
                # 質問パターンのチェック
                if any(pattern in sentence for pattern in question_markers['patterns']):
                    return True
                return False
            
            # 各文を評価
            questions = sum(1 for s in sentences if is_question(s))
            total_sentences = len([s for s in sentences if s.strip()])
            ratio = questions / total_sentences if total_sentences else 0
            
            # 理想の比率(0.3)からの距離に基づいてスコアを計算
            balance_score = max(0.0, 1.0 - min(abs(0.3 - ratio), 0.2) * 2)
            
            # 2. Sentence length change
            lengths = [len(s.strip()) for s in sentences if s.strip()]
            length_variance = np.var(lengths) if len(lengths) > 1 else 0
            length_score = 1.0 / (1.0 + length_variance/500)  # Higher score if variance is small
            
            # 3. Use of conjunctions
            conjunctions = ['しかし', 'だから', 'また', 'そして', 'したがって']
            conj_count = sum(1 for s in sentences if any(c in s for c in conjunctions))
            conj_ratio = conj_count / len(sentences)
            conj_score = min(1.0, conj_ratio * 2)  # Evaluate moderate usage
            
            # Weighted average of scores
            weights = [0.5, 0.25, 0.25]  # Balance, length, conjunction weights
            final_score = sum(s * w for s, w in zip([balance_score, length_score, conj_score], weights))
            
            return max(0.1, min(1.0, final_score))  # Limit to range 0.1 to 1.0
        
        flow_scores = [calculate_dialogue_flow(pred) for pred in decoded_preds]
        
        style_score = np.mean(style_scores)
        flow_score = np.mean(flow_scores)
        
        # Add overall evaluation score
        combined_score = (style_score * 0.6 + flow_score * 0.4)  # Increase flow_score weight
        
        return {
            'style_consistency': style_score,
            'dialogue_flow': flow_score,
            'combined_score': combined_score
        }




<style>
pre {
    border: 1px solid #333;
    padding: 20px;
    margin: 20px 0;
    background-color: #000000;
    color: #d4d4d4;
    border-radius: 8px;
}
pre code {
    color: #d4d4d4;
    display: block;
    padding-bottom: 8px;
    background-color: #000000; 
}

.hljs, .language-python {
    background-color: #000000 !important;
}
</style>

<div style="background-color: #F9F4F0; padding: 10px; border-left: 5px solid #4CAF50; margin: 10px; width: 95%;">
    <details>
        <summary style="color: #8A6F5C; font-size: 1.17em; font-weight: bold;">claude解説</summary>
        <div style="color: #8A6F5C;">
このコードについて、ソクラテス式対話チャットボットの文脈で説明させていただきます。

# 評価メトリクスの概要

このコードは、チャットボットの応答の質を評価する仕組みを実装しています。主に以下の2つの側面から評価を行っています：

1. スタイルの一貫性 (`style_consistency`)
2. 対話の流れ (`dialogue_flow`)

## スタイルの一貫性の評価

```python
sentence_end_patterns = {
    'question_patterns': [
        'かね', 'だろうか', 'ではないか',
        'のか', 'と思わないか', '考えてみよう',
    ],
    # ...
}
```

### 具体例：
元の応答：
```
その考えは興味深いですね。しかし、なぜそのように考えるのでしょうか？私たちは、この問題についてもう少し深く考えてみる必要があるのではないでしょうか。
```

このような応答は以下の点で高評価となります：
- 「興味深い」という反応的表現の使用
- 「のでしょうか」という質問形式
- 「ではないでしょうか」というソクラテス式の問いかけ

## 対話の流れの評価

```python
def calculate_dialogue_flow(text):
    # 質問と説明のバランスを評価
    ratio = questions / total_sentences
    # 理想の比率(0.3 = 30%が質問文)を目指す
```

### 具体例：
良い応答の例：
```
その点について、私も考えを巡らせていました（説明）。
しかし、そもそもなぜその前提に立つのでしょうか？（質問）
もし別の角度から見たとすれば、どのような可能性が見えてくるでしょうか？（質問）
```

この応答は以下の理由で高評価となります：
- 質問と説明のバランスが良い（約30%が質問文）
- 「しかし」という接続詞の適切な使用
- 文の長さに極端な差がない

## 評価の重み付け

```python
combined_score = (style_score * 0.6 + flow_score * 0.4)
```

最終的な評価は：
- スタイルの一貫性（60%）
  - 文末表現の一貫性（50%）
  - 助動詞の使用（30%）
  - 文長の一貫性（20%）
- 対話の流れ（40%）
  - 質問と説明のバランス（50%）
  - 文の長さの変化（25%）
  - 接続詞の使用（25%）

### 具体例：
低評価となる応答：
```
はい。そうですね。わかりました。
```
理由：
- 文末表現が単調
- 質問がない
- 接続詞がない
- 文が短すぎる

高評価となる応答：
```
なるほど、その視点は興味深いですね。
しかし、その考えの根底にある前提について、もう少し掘り下げて考えてみましょうか。
たとえば、この状況を別の角度から見たとき、どのような可能性が見えてくるのでしょうか。
```
理由：
- 文末表現が多様（「ですね」「みましょうか」「でしょうか」）
- 適度な質問の含有率
- 接続詞の適切な使用（「しかし」「たとえば」）
- 文の長さのバランスが良い


        
</div>
    </details>
</div>


In [ ]:
# Add memory usage monitoring log
def log_memory_usage():
    import psutil
    import torch
    
    # CPU memory
    process = psutil.Process()
    cpu_memory = process.memory_info().rss / 1024 / 1024  # MB
    
    # GPU memory
    gpu_memory = []
    if torch.cuda.is_available():
        for i in range(torch.cuda.device_count()):
            gpu_memory.append({
                'device': i,
                'allocated': torch.cuda.memory_allocated(i) / 1024 / 1024,  # MB
                'reserved': torch.cuda.memory_reserved(i) / 1024 / 1024,    # MB
                'max_allocated': torch.cuda.max_memory_allocated(i) / 1024 / 1024  # MB
            })
    
    logging.info(f"CPU Memory usage: {cpu_memory:.2f} MB")
    for gpu in gpu_memory:
        logging.info(f"GPU {gpu['device']} Memory:")
        logging.info(f"  - Allocated: {gpu['allocated']:.2f} MB")
        logging.info(f"  - Reserved: {gpu['reserved']:.2f} MB")
        logging.info(f"  - Max Allocated: {gpu['max_allocated']:.2f} MB")




<style>
pre {
    border: 1px solid #333;
    padding: 20px;
    margin: 20px 0;
    background-color: #000000;
    color: #d4d4d4;
    border-radius: 8px;
}
pre code {
    color: #d4d4d4;
    display: block;
    padding-bottom: 8px;
    background-color: #000000; 
}

.hljs, .language-python {
    background-color: #000000 !important;
}
</style>

<div style="background-color: #F9F4F0; padding: 10px; border-left: 5px solid #4CAF50; margin: 10px; width: 95%;">
    <details>
        <summary style="color: #8A6F5C; font-size: 1.17em; font-weight: bold;">claude解説</summary>
        <div style="color: #8A6F5C;">

このコードはメモリ使用状況を監視・記録するための関数について説明します。

# メモリ監視関数の解説

```python
def log_memory_usage():
```

この関数は、CPUとGPUのメモリ使用状況を監視し、ログに記録します。

## 1. CPU メモリの監視

```python
# CPU memory
process = psutil.Process()
cpu_memory = process.memory_info().rss / 1024 / 1024  # MB
```

- `psutil.Process()`：現在実行中のPythonプロセスを取得
- `rss`（Resident Set Size）：プロセスが使用している実メモリ量
- 単位変換：バイトからメガバイト（MB）に変換（1024で2回割る）

### 例：
```
CPU Memory usage: 8542.45 MB
```
これは約8.5GBのメモリを使用していることを示します。

## 2. GPU メモリの監視

```python
# GPU memory
gpu_memory = []
if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
```

GPUが利用可能な場合、各GPUについて以下の情報を収集：

1. **Allocated Memory（割り当て済みメモリ）**
```python
'allocated': torch.cuda.memory_allocated(i) / 1024 / 1024
```
- 実際にPyTorchが使用中のGPUメモリ量
- 例：モデルの重みやテンソルが使用中のメモリ

2. **Reserved Memory（予約済みメモリ）**
```python
'reserved': torch.cuda.memory_reserved(i) / 1024 / 1024
```
- PyTorchが確保しているが、まだ使用していないメモリ量
- キャッシュとして確保されている領域

3. **Max Allocated Memory（最大割り当てメモリ）**
```python
'max_allocated': torch.cuda.max_memory_allocated(i) / 1024 / 1024
```
- プログラム開始からの最大メモリ使用量
- メモリリークの検出に有用

### 出力例：
```
GPU 0 Memory:
  - Allocated: 3584.25 MB
  - Reserved: 4096.00 MB
  - Max Allocated: 3842.12 MB

GPU 1 Memory:
  - Allocated: 3621.83 MB
  - Reserved: 4096.00 MB
  - Max Allocated: 3912.45 MB
```

## 実用的な使用例

このソクラテス式対話モデルの学習では、以下のような場面で特に重要です：

1. **モデルロード時**
```python
# モデルロード前後でメモリ使用量を確認
log_memory_usage()  # Before
model = AutoModelForCausalLM.from_pretrained(...)
log_memory_usage()  # After
```

2. **バッチ処理時**
```python
# 大きなバッチを処理する前後でメモリをチェック
log_memory_usage()  # Before batch
# バッチ処理
log_memory_usage()  # After batch
```

3. **メモリリーク検出**
```python
# 定期的なメモリ使用量の記録
for epoch in range(epochs):
    log_memory_usage()  # 各エポックの開始時
    # 学習処理
    log_memory_usage()  # 各エポックの終了時
```

このモニタリングにより、以下のような問題を早期に発見できます：
- メモリリーク
- GPUメモリの不均衡な使用
- 予期せぬメモリ消費の急増

これは特に大規模な言語モデルの学習において重要で、メモリ関連の問題を事前に検出し、学習の安定性を確保するのに役立ちます。

        
</div>
    </details>
</div>


In [ ]:
# Add memory cleanup
def clear_memory():
    gc.collect()
    torch.cuda.empty_cache()
    


<style>
pre {
    border: 1px solid #333;
    padding: 20px;
    margin: 20px 0;
    background-color: #000000;
    color: #d4d4d4;
    border-radius: 8px;
}
pre code {
    color: #d4d4d4;
    display: block;
    padding-bottom: 8px;
    background-color: #000000; 
}

.hljs, .language-python {
    background-color: #000000 !important;
}
</style>

<div style="background-color: #F9F4F0; padding: 10px; border-left: 5px solid #4CAF50; margin: 10px; width: 95%;">
    <details>
        <summary style="color: #8A6F5C; font-size: 1.17em; font-weight: bold;">claude解説</summary>
        <div style="color: #8A6F5C;">



# メモリクリーンアップ関数の解説

```python
def clear_memory():
    gc.collect()
    torch.cuda.empty_cache()
```

この関数は、CPUとGPUのメモリを解放するためのシンプルだが重要な機能を提供します。

## 1. `gc.collect()`

```python
gc.collect()
```

- **目的**: Pythonのガベージコレクション（不要なメモリの解放）を手動で実行
- **動作**: 参照されなくなったPythonオブジェクトを検出し、メモリから解放
- **重要性**: 大規模な言語モデルの学習では、大量のテンソルや中間データが生成されるため、定期的なクリーンアップが必要

### 使用例：
```python
# 大きなデータ処理後のクリーンアップ
del large_dataset  # 明示的な削除
clear_memory()     # メモリの解放
```

## 2. `torch.cuda.empty_cache()`

```python
torch.cuda.empty_cache()
```

- **目的**: GPUのキャッシュメモリを解放
- **動作**: PyTorchが予約しているが使用していないGPUメモリを解放
- **重要性**: GPU メモリの効率的な使用のため、特に長時間の学習セッションで重要

### 使用例：
```python
# バッチ処理後のGPUメモリクリーンアップ
del batch_output  # テンソルの削除
clear_memory()    # キャッシュの解放
```

## 実践的な使用シーン

### 1. エポック間のクリーンアップ
```python
for epoch in range(epochs):
    # 学習処理
    trainer.train()
    
    # エポック終了時のクリーンアップ
    clear_memory()
    log_memory_usage()  # メモリ使用状況の確認
```

### 2. 大きなバッチ処理後
```python
# 大きなバッチの処理
outputs = model(large_batch)
loss = outputs.loss
loss.backward()

# メモリの解放
del outputs
del loss
clear_memory()
```

### 3. モデルの切り替え時
```python
# 古いモデルの解放
del old_model
clear_memory()

# 新しいモデルのロード
new_model = AutoModelForCausalLM.from_pretrained(...)
```

## メモリ管理の重要性

ソクラテス式対話モデルの学習では、以下の理由でメモリ管理が特に重要です：

1. **大規模なモデルサイズ**
   - 基本モデル（Gemma 2B）自体が大きい
   - LoRAパラメータの追加
   - 中間層の活性化値

2. **長い系列長**
   - 対話形式のため、入力テキストが長くなりやすい
   - アテンションメカニズムのメモリ要求

3. **バッチ処理**
   - 複数の対話サンプルを同時に処理
   - グラデーント蓄積

### メモリ問題の例と対処：

```python
try:
    # 大きなバッチの処理
    outputs = model(large_batch)
except RuntimeError as e:
    if "out of memory" in str(e):
        # メモリ不足時の対処
        clear_memory()
        # バッチサイズを半分に
        outputs = model(large_batch[:len(large_batch)//2])
```

このような適切なメモリ管理により：
- 学習の安定性が向上
- OOMエラー（Out of Memory）を防止
- より効率的なGPUリソースの使用
が可能になります。

        
</div>
    </details>
</div>


In [ ]:
# Update custom callbacks
class StyleCallback(TrainerCallback):
    def __init__(self):
        self.style_scores = []
        self.flow_scores = []
        
    def on_evaluate(self, args, state, control, metrics, **kwargs):
        if 'eval_style_consistency' in metrics:
            self.style_scores.append(metrics['eval_style_consistency'])
            self.flow_scores.append(metrics['eval_dialogue_flow'])
            
            # Log detailed information
            logging.info(f"Step {state.global_step}:")
            logging.info(f"Style Consistency: {metrics['eval_style_consistency']:.3f}")
            logging.info(f"Dialogue Flow: {metrics['eval_dialogue_flow']:.3f}")
    
    def on_train_end(self, args, state, control, **kwargs):
        # Log overall evaluation
        avg_style = sum(self.style_scores) / len(self.style_scores) if self.style_scores else 0
        avg_flow = sum(self.flow_scores) / len(self.flow_scores) if self.flow_scores else 0
        
        logging.info("Training Complete!")
        logging.info(f"Average Style Consistency: {avg_style:.3f}")
        logging.info(f"Average Dialogue Flow: {avg_flow:.3f}")



<style>
pre {
    border: 1px solid #333;
    padding: 20px;
    margin: 20px 0;
    background-color: #000000;
    color: #d4d4d4;
    border-radius: 8px;
}
pre code {
    color: #d4d4d4;
    display: block;
    padding-bottom: 8px;
    background-color: #000000; 
}

.hljs, .language-python {
    background-color: #000000 !important;
}
</style>

<div style="background-color: #F9F4F0; padding: 10px; border-left: 5px solid #4CAF50; margin: 10px; width: 95%;">
    <details>
        <summary style="color: #8A6F5C; font-size: 1.17em; font-weight: bold;">claude解説</summary>
        <div style="color: #8A6F5C;">



# StyleCallbackクラスの解説

このクラスは、学習中のモデルのスタイル一貫性と対話の流れを監視するためのカスタムコールバックです。

## クラスの基本構造

```python
class StyleCallback(TrainerCallback):
    def __init__(self):
        self.style_scores = []  # スタイル一貫性スコアを保存
        self.flow_scores = []   # 対話の流れスコアを保存
```

## 1. 評価時の処理 (`on_evaluate`)

```python
def on_evaluate(self, args, state, control, metrics, **kwargs):
    if 'eval_style_consistency' in metrics:
        self.style_scores.append(metrics['eval_style_consistency'])
        self.flow_scores.append(metrics['eval_dialogue_flow'])
```

### 具体例：

評価時のログ出力：
```
Step 100:
Style Consistency: 0.856
Dialogue Flow: 0.742
```

これは以下を示します：
- 学習の100ステップ目
- スタイル一貫性スコア: 85.6%
  - 例：「なるほど」「考えてみましょうか」などのソクラテス式の表現が適切に使用されている
- 対話の流れスコア: 74.2%
  - 例：質問と説明のバランスが良好

## 2. 学習終了時の処理 (`on_train_end`)

```python
def on_train_end(self, args, state, control, **kwargs):
    avg_style = sum(self.style_scores) / len(self.style_scores) if self.style_scores else 0
    avg_flow = sum(self.flow_scores) / len(self.flow_scores) if self.flow_scores else 0
```

### 具体例：

学習終了時のログ出力：
```
Training Complete!
Average Style Consistency: 0.823
Average Dialogue Flow: 0.751
```

これは全学習期間を通じての平均スコアを示します：
- 平均スタイル一貫性: 82.3%
- 平均対話の流れ: 75.1%

## 実際の使用例

```python
# トレーナーの設定
trainer = Trainer(
    model=model,
    args=training_args,
    callbacks=[StyleCallback()],  # コールバックの追加
    # ...
)

# 学習中の出力例：
"""
Step 50:
Style Consistency: 0.721
Dialogue Flow: 0.689

Step 100:
Style Consistency: 0.856
Dialogue Flow: 0.742

...

Training Complete!
Average Style Consistency: 0.823
Average Dialogue Flow: 0.751
"""
```

## スコアの解釈

### スタイル一貫性スコア (Style Consistency)
- **高スコア（0.8以上）の例**：
```
なるほど、その考えは興味深いですね。
しかし、その前提について、もう少し掘り下げて考えてみましょうか？
```

- **低スコア（0.5以下）の例**：
```
はい、そうです。
わかりました。
次に進みましょう。
```

### 対話の流れスコア (Dialogue Flow)
- **高スコア（0.8以上）の例**：
```
その視点は確かに重要ですね。
しかし、ここで一つ質問させていただきたいのですが、
なぜそのような結論に至ったのでしょうか？
```

- **低スコア（0.5以下）の例**：
```
それは違います。
なぜですか？
どうしてですか？
どう思いますか？
```

## 活用方法

1. **学習の進捗モニタリング**
   - 定期的なスコアの確認
   - 急激なスコアの低下を検知

2. **モデルの改善**
   - スコアの傾向から問題点を特定
   - 必要に応じてハイパーパラメータを調整

3. **最終評価**
   - モデルの全体的な性能を評価
   - 異なるモデルバージョンの比較

このコールバックにより、モデルがソクラテス式の対話スタイルを適切に学習できているかを定量的に評価することができます。

        
</div>
    </details>
</div>


In [ ]:

# Extend custom callbacks
class TrainingMonitorCallback(TrainerCallback):
    def __init__(self):
        # Import psutil here as well for safety
        import psutil
        self.train_start_time = None
        self.metrics_history = {
            'step': [],
            'style_consistency': [],
            'dialogue_flow': [],
            'combined_score': [],
            'loss': [],
            'learning_rate': [],
            'epoch': [],
            'cpu_ram_usage': [],
            'gpu_vram_usage': [],
            'gpu_utilization': [],
            'batch_size': [],
            'moving_avg_loss': [],
            # 新しい詳細メトリクス
            'lr_schedule': [],
            'batch_metrics': [],
            'gpu_metrics': [],
            'grad_norm': []
        }
        self.peak_metrics = {
            'cpu_ram': 0,
            'gpu_vram': 0,
            'gpu_util': 0
        }
        self.output_dir = Path(f"{BASE_OUTPUT_DIR}/training_progress")
        self.output_dir.mkdir(parents=True, exist_ok=True)
        
    def _record_resource_usage(self):
        """Record current resource usage with timestamp"""
        import psutil
        import torch
        current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        
        # CPU RAM
        cpu_ram = psutil.Process().memory_info().rss / (1024 * 1024 * 1024)  # GB
        self.peak_metrics['cpu_ram'] = max(self.peak_metrics['cpu_ram'], cpu_ram)
        
        # GPU metrics with timestamp
        if torch.cuda.is_available():
            gpu_metrics = []
            for i in range(torch.cuda.device_count()):
                vram_used = torch.cuda.memory_allocated(i) / (1024 * 1024 * 1024)  # GB
                self.peak_metrics['gpu_vram'] = max(self.peak_metrics['gpu_vram'], vram_used)
                
                # GPU utilization (requires nvidia-smi)
                try:
                    import subprocess
                    result = subprocess.check_output(['nvidia-smi', '--query-gpu=utilization.gpu', '--format=csv,noheader,nounits'])
                    gpu_util = float(result.decode('utf-8').strip())
                    self.peak_metrics['gpu_util'] = max(self.peak_metrics['gpu_util'], gpu_util)
                except:
                    gpu_util = 0
                
                gpu_metrics.append({
                    'device': i,
                    'vram_used': vram_used,
                    'utilization': gpu_util
                })
                
            # 時系列データとして保存
            self.metrics_history['gpu_metrics'].append({
                'timestamp': current_time,
                'metrics': gpu_metrics
            })
                
            self.metrics_history['cpu_ram_usage'].append(cpu_ram)
            self.metrics_history['gpu_vram_usage'].append(vram_used)
            self.metrics_history['gpu_utilization'].append(gpu_util)
    
    def on_train_begin(self, args, state, control, **kwargs):
        self.train_start_time = datetime.now()
        logging.info("Training started at: %s", self.train_start_time)
        self._record_resource_usage()
        
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs:
            current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            
            # 学習率とスケジューリングの記録
            if 'learning_rate' in logs:
                self.metrics_history['lr_schedule'].append({
                    'timestamp': current_time,
                    'step': state.global_step,
                    'learning_rate': logs['learning_rate'],
                    'schedule_type': args.lr_scheduler_type
                })
                self.metrics_history['learning_rate'].append(logs['learning_rate'])
            
            # バッチサイズと損失値の関連を記録
            if 'loss' in logs:
                self.metrics_history['batch_metrics'].append({
                    'timestamp': current_time,
                    'step': state.global_step,
                    'batch_size': args.per_device_train_batch_size,
                    'loss': logs['loss'],
                    'grad_norm': logs.get('grad_norm', None)
                })
                self.metrics_history['loss'].append(logs['loss'])
                self.metrics_history['batch_size'].append(args.per_device_train_batch_size)
                if 'grad_norm' in logs:
                    self.metrics_history['grad_norm'].append(logs['grad_norm'])
            
            # 移動平均の計算と記録
            if len(self.metrics_history['loss']) > 10:
                avg_loss = sum(self.metrics_history['loss'][-10:]) / 10
                self.metrics_history['moving_avg_loss'].append(avg_loss)
                logging.info(f"Moving average loss (last 10 steps): {avg_loss:.4f}")
            
            logging.info(f"Step {state.global_step}: {logs}")
            if 'grad_norm' in logs:
                logging.info(f"Gradient norm: {logs['grad_norm']:.4f}")
            
        self._record_resource_usage()
        
    def on_train_end(self, args, state, control, **kwargs):
        training_duration = datetime.now() - self.train_start_time
        
        # 詳細な学習履歴の保存
        training_history = {
            'lr_schedule': self.metrics_history['lr_schedule'],
            'batch_metrics': self.metrics_history['batch_metrics'],
            'gpu_metrics': self.metrics_history['gpu_metrics'],
            'moving_avg_loss': self.metrics_history['moving_avg_loss']
        }
        
        # 学習履歴をJSONファイルとして保存
        history_file = self.output_dir / 'training_history.json'
        with open(history_file, 'w', encoding='utf-8') as f:
            json.dump(training_history, f, indent=2, ensure_ascii=False)
        
        # 基本的なメトリクスのログ出力
        logging.info(f"Training completed. Total duration: {training_duration}")
        logging.info(f"Peak CPU RAM usage: {self.peak_metrics['cpu_ram']:.2f} GB")
        logging.info(f"Peak GPU VRAM usage: {self.peak_metrics['gpu_vram']:.2f} GB")
        logging.info(f"Peak GPU utilization: {self.peak_metrics['gpu_util']:.1f}%")
        
        # 最終サマリーの作成と保存
        summary = {
            'training_duration': str(training_duration),
            'final_loss': self.metrics_history['loss'][-1] if self.metrics_history['loss'] else None,
            'best_combined_score': max(filter(None, self.metrics_history['combined_score'])) if self.metrics_history['combined_score'] else None,
            'total_steps': len(self.metrics_history['step']),
            'final_epoch': self.metrics_history['epoch'][-1] if self.metrics_history['epoch'] else None,
            'learning_rate_summary': {
                'initial': self.metrics_history['learning_rate'][0] if self.metrics_history['learning_rate'] else None,
                'final': self.metrics_history['learning_rate'][-1] if self.metrics_history['learning_rate'] else None,
                'schedule_type': args.lr_scheduler_type
            },
            'loss_summary': {
                'final_moving_avg': self.metrics_history['moving_avg_loss'][-1] if self.metrics_history['moving_avg_loss'] else None,
                'best_loss': min(self.metrics_history['loss']) if self.metrics_history['loss'] else None
            },
            'resource_usage': {
                'peak_cpu_ram_gb': self.peak_metrics['cpu_ram'],
                'peak_gpu_vram_gb': self.peak_metrics['gpu_vram'],
                'peak_gpu_utilization': self.peak_metrics['gpu_util']
            },
            'hardware_info': {
                'cpu_info': self._get_cpu_info(),
                'gpu_info': self._get_gpu_info(),
                'total_ram': self._get_total_ram()
            }
        }
        
        # サマリーをJSONファイルとして保存
        with open(self.output_dir / 'training_summary.json', 'w', encoding='utf-8') as f:
            json.dump(summary, f, indent=2, ensure_ascii=False)
            
        logging.info("Training Complete!")
        logging.info(f"Training duration: {summary['training_duration']}")
        
        # Noneチェックを追加
        if summary['loss_summary']['final_moving_avg'] is not None:
            logging.info(f"Final moving average loss: {summary['loss_summary']['final_moving_avg']:.4f}")
        if summary['loss_summary']['best_loss'] is not None:
            logging.info(f"Best loss achieved: {summary['loss_summary']['best_loss']:.4f}")
        
        logging.info(f"Peak CPU RAM usage: {summary['resource_usage']['peak_cpu_ram_gb']:.2f} GB")
        logging.info(f"Peak GPU VRAM usage: {summary['resource_usage']['peak_gpu_vram_gb']:.2f} GB")
        logging.info(f"Peak GPU utilization: {summary['resource_usage']['peak_gpu_utilization']:.1f}%")

    def _get_cpu_info(self):
        import cpuinfo
        try:
            info = cpuinfo.get_cpu_info()
            return {
                'model': info.get('brand_raw', 'Unknown'),
                'cores': psutil.cpu_count(logical=False),
                'threads': psutil.cpu_count(logical=True)
            }
        except:
            return "Failed to get CPU info"
            
    def _get_gpu_info(self):
        if not torch.cuda.is_available():
            return "No GPU available"
        try:
            import subprocess
            result = subprocess.check_output(['nvidia-smi', '--query-gpu=gpu_name,memory.total', '--format=csv,noheader,nounits'])
            gpus = result.decode('utf-8').strip().split('\n')
            return [{'model': g.split(',')[0], 'memory': float(g.split(',')[1])/1024} for g in gpus]
        except:
            return "Failed to get GPU info"
            
    def _get_total_ram(self):
        return psutil.virtual_memory().total / (1024 * 1024 * 1024)  # GB



<style>
pre {
    border: 1px solid #333;
    padding: 20px;
    margin: 20px 0;
    background-color: #000000;
    color: #d4d4d4;
    border-radius: 8px;
}
pre code {
    color: #d4d4d4;
    display: block;
    padding-bottom: 8px;
    background-color: #000000; 
}

.hljs, .language-python {
    background-color: #000000 !important;
}
</style>

<div style="background-color: #F9F4F0; padding: 10px; border-left: 5px solid #4CAF50; margin: 10px; width: 95%;">
    <details>
        <summary style="color: #8A6F5C; font-size: 1.17em; font-weight: bold;">claude解説</summary>
        <div style="color: #8A6F5C;">



# TrainingMonitorCallbackの詳細解説

このクラスは、学習プロセスの詳細な監視と記録を行う高度なコールバックシステムです。

## 1. 初期化と構造

```python
def __init__(self):
    self.metrics_history = {
        'step': [],
        'style_consistency': [],
        'dialogue_flow': [],
        # ... その他のメトリクス
    }
```

### 記録される主要メトリクス：
- 学習の進捗（ステップ、エポック）
- モデルの性能（スタイル一貫性、対話の流れ）
- リソース使用状況（CPU/GPU使用率）
- 学習パラメータ（学習率、バッチサイズ）

## 2. リソース使用状況の記録

```python
def _record_resource_usage(self):
    # CPU RAM監視
    cpu_ram = psutil.Process().memory_info().rss / (1024 * 1024 * 1024)
    
    # GPU監視
    if torch.cuda.is_available():
        vram_used = torch.cuda.memory_allocated(i) / (1024 * 1024 * 1024)
```

### 出力例：
```
Resource Usage at 2024-03-15 14:30:25:
CPU RAM: 12.45 GB
GPU 0: 
  - VRAM Used: 8.32 GB
  - Utilization: 85%
```

## 3. 学習プロセスの監視

### 学習開始時
```python
def on_train_begin(self, args, state, control, **kwargs):
    self.train_start_time = datetime.now()
```

### ログ記録時
```python
def on_log(self, args, state, control, logs=None, **kwargs):
    # 学習率の記録
    if 'learning_rate' in logs:
        self.metrics_history['lr_schedule'].append({
            'timestamp': current_time,
            'step': state.global_step,
            'learning_rate': logs['learning_rate']
        })
```

### 出力例：
```
Step 100:
Learning Rate: 7.5e-5
Loss: 2.345
Moving Average Loss (last 10 steps): 2.412
Gradient Norm: 0.876
```

## 4. 学習終了時のサマリー生成

```python
def on_train_end(self, args, state, control, **kwargs):
    summary = {
        'training_duration': str(training_duration),
        'final_loss': self.metrics_history['loss'][-1],
        'best_combined_score': max(self.metrics_history['combined_score']),
        # ... その他の統計
    }
```

### サマリー出力例：
```json
{
    "training_duration": "5:23:45",
    "final_loss": 1.234,
    "best_combined_score": 0.856,
    "resource_usage": {
        "peak_cpu_ram_gb": 14.5,
        "peak_gpu_vram_gb": 10.2,
        "peak_gpu_utilization": 92.5
    },
    "learning_rate_summary": {
        "initial": 8e-5,
        "final": 1.2e-5
    }
}
```

## 5. 実際の使用例

```python
# トレーナーの設定
trainer = Trainer(
    model=model,
    args=training_args,
    callbacks=[
        TrainingMonitorCallback(),
        StyleCallback()
    ]
)

# 学習実行
trainer.train()

# 出力ディレクトリの構造
training_progress/
├── training_history.json  # 詳細な学習履歴
└── training_summary.json  # 最終サマリー
```

## 6. モニタリングの活用方法

### 1. リソース使用の最適化
```python
# メモリ使用量が閾値を超えた場合の警告
if self.peak_metrics['gpu_vram'] > 10:  # 10GB
    logging.warning("High GPU memory usage detected!")
```

### 2. 学習の進捗モニタリング
```python
# 損失値の急激な変化の検出
if len(self.metrics_history['loss']) > 1:
    current_loss = self.metrics_history['loss'][-1]
    prev_loss = self.metrics_history['loss'][-2]
    if current_loss > prev_loss * 1.5:  # 50%以上の増加
        logging.warning("Significant loss increase detected!")
```

### 3. ハードウェアの状態監視
```python
# GPU使用率の監視
if gpu_util > 95:  # 95%以上の使用率
    logging.warning("GPU utilization is very high!")
```

このコールバックは、ソクラテス式対話モデルの学習において：
- メモリ使用の最適化
- 学習の安定性確保
- パフォーマンスのボトルネック検出
- 問題の早期発見と対処
を可能にし、効率的な学習プロセスの実現を支援します。

        
</div>
    </details>
</div>
